In [ ]:
! pip install "confluent-kafka[avro]"

In [ ]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.0.2 pyspark-shell'

from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils
from confluent_kafka.avro.cached_schema_registry_client import CachedSchemaRegistryClient
from confluent_kafka.avro.serializer.message_serializer import MessageSerializer

In [ ]:
sc = SparkContext()
sc.setLogLevel("ERROR")

In [ ]:
ssc = StreamingContext(sc, 20)

In [ ]:
schema_registry_client = CachedSchemaRegistryClient(url='http://schema-registry:8081')
serializer = MessageSerializer(schema_registry_client)

In [ ]:
# Decoder avro
def decoder(s):
    decoded_message = serializer.decode_message(s)
    return decoded_message

In [ ]:
kafkaStream = KafkaUtils.createStream(ssc,
                                          'zookeeper:32181',
                                          'spark-streaming',
                                          {'kafka-music-charts-song-play-count-repartition':1},
                                          valueDecoder=serializer.decode_message)

In [ ]:
lines = kafkaStream.map(lambda x: x[1])
lines.pprint()

In [ ]:
ssc.start()
ssc.awaitTermination()